In [5]:
from pysb import *
import pandas as pd
from pysb.core import *
from pysb.bng import *
from pysb.integrate import *
import matplotlib.pyplot as plt
import numpy as np
import scipy.interpolate

Model ()

/Users/geenaildefonso/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: ModelExistsWarning: Redefining model! (You can probably ignore this if you are running code interactively)


<Model '_interactive_' (monomers: 0, rules: 0, parameters: 0, expressions: 0, compartments: 0) at 0x1199bbcd0>

In [6]:
#Declaration of monomers

""" Delcares IkBa, IkBb, IkBe, and IkBd monomers. All four of these monomers have binding sites to IKK2
    (for IkBa, IkBb, IkBe), and IKK1 (for IkBd). The monomers import and export between the Cytoplasm
    and the Nucleus.  Declares the IkBa, IkBb, IkBe, and IkBd mRNA monomers. The monomers go through constitutive 
    RNA synthesis without the presence of NFkB in the nucleus,  and inducible RNA synthesis in the presence of NFkB
    in the nucleus. """

# def ikba_and_mRNA_monomers():
Monomer('IkBa', ['ikk', 'nfkb', 'S'], {'S': ['C', 'N']})
Monomer('IkBa_mRNA')

# def ikbb_and_mRNA_monomers():
Monomer('IkBb', ['ikk', 'nfkb', 'S'], {'S': ['C', 'N']})
Monomer('IkBb_mRNA')

# def ikbe_and_mRNA_monomers():
Monomer('IkBe', ['ikk', 'nfkb', 'S'], {'S': ['C', 'N']})
Monomer('IkBe_mRNA')

# def ikbd_and_mRNA_monomers():
Monomer('IkBd', ['ikk', 'nfkb', 'S'], {'S': ['C', 'N']})
Monomer('IkBd_mRNA')


# Declares NFkB, IKK1, and IKK2 which having a binding site to IkB and NFkB can transport between the Cytoplasm
# and the Nucleus. IKK1 and IKK2 only exist in the Cytoplasm.

# def nfkb_and_ikk_monomers():
Monomer('NFkB', ['ikb', 'S'], {'S': ['C', 'N']})

# def ikk1_monomer():
Monomer('IKK1', ['ikb', 'S'], {'S': ['C', 'N']})

# def ikk2_monomer():
Monomer('IKK2', ['ikb', 'S'], {'S': ['C', 'N']})

Monomer('IKK2', ['ikb', 'S'], {'S': ['C', 'N']})

In [7]:
#Declaration of initial conditions
# def initial_ikk1_conditions():
Parameter('IKK1_0', 0.1) #Inhibitor Kinase Kinase a
Initial(IKK1(ikb=None, S='C'), IKK1_0)

# def initial_ikk2_conditions():
Parameter('IKK2_0', 0.1) #Inhibitor Kinase Kinase b
Initial(IKK2(ikb=None, S='C'), IKK2_0)

# def initial nfkb_conditions():
Parameter('NFkB_0', 0.125) #Nuclear Factor-kappaB
Initial(NFkB(ikb=None, S='N'), NFkB_0)

#RNA synthesis by NFkBn and Hill Coefficient

# Observable('IkBa_mRNA_obs', IkBa_mRNA())
# Observable('IkBa_obs', IkBa(b=ANY, c=ANY, S=ANY))

# Observable('NFkBn_obs', NFkB(ikb=WILD, S='N'))
Observable('NFkBn_free', NFkB(ikb=None, S='N'))
# Observable('NFkBn_bound', NFkB(ikb=ANY, S='N'))
Observable('IKK2_obs', IKK2())

Observable('IKK2_obs', IKK2())

In [8]:
Parameter('hill', 3)
Parameter('a', 6)
Parameter('b', 0.25)
Parameter('e', 0.5)
Parameter('d', 0.025)

Expression('a_NFkBn', a*(NFkBn_free)**(hill))
Expression('b_NFkBn', b*(NFkBn_free)**(hill))
Expression('e_NFkBn', e*(NFkBn_free)**(hill))
Expression('d_NFkBn', d*(NFkBn_free)**(hill))

Rule('an_mRNA', None >> IkBa_mRNA(), a_NFkBn)
Rule('bn_mRNA', None >> IkBb_mRNA(), b_NFkBn)
Rule('en_mRNA', None >> IkBe_mRNA(), e_NFkBn)
Rule('dn_mRNA', None >> IkBd_mRNA(), d_NFkBn)

# IkB mRNA and protein synthesis reactions

Parameter('mRNA_a', 0.035)
Rule('a_mRNA', IkBa_mRNA() >> None, mRNA_a)


Parameter('mRNA_b', 3e-3)
Parameter('mRNA_e', 4e-3)
Parameter('mRNA_d', 2e-3)

Rule('b_mRNA', IkBb_mRNA() >> None, mRNA_b)
Rule('e_mRNA', IkBe_mRNA() >> None, mRNA_e)
Rule('d_mRNA', IkBd_mRNA() >> None, mRNA_d)

Rule('d_mRNA', IkBd_mRNA() >> None, mRNA_d)

In [9]:
Parameter('syntha', 0.2448)
Rule('a_psynth', IkBa_mRNA() >> IkBa(ikk=None, nfkb=None, S='C') + IkBa_mRNA(), syntha)

Parameter('synthb', 0.2448)
Rule('b_psynth', IkBb_mRNA() >> IkBb(ikk=None, nfkb=None, S='C') + IkBb_mRNA(), synthb)

Parameter('synthe', 0.2448)
Rule('e_psynth', IkBe_mRNA() >> IkBe(ikk=None, nfkb=None, S='C') + IkBe_mRNA(), synthe)

Parameter('synthd', 0.2448)
Rule('d_psynth', IkBd_mRNA() >> IkBd(ikk=None, nfkb=None, S='C') + IkBd_mRNA(), synthd)


Parameter('psynth_a', 2e-4)
Parameter('psynth_b', 1e-5)
Parameter('psynth_e', 3e-6)
Parameter('psynth_d', 1e-7)
Rule('a_synth', None >> IkBa_mRNA(), psynth_a)
Rule('b_synth', None >> IkBb_mRNA(), psynth_b)
Rule('e_synth', None >> IkBe_mRNA(), psynth_e)
Rule('d_synth', None >> IkBd_mRNA(), psynth_d)


Rule('d_synth', None >> IkBd_mRNA(), psynth_d)

In [10]:
#IkB(a,b,e) association and dissociation from IKK2 and IkBd association and dissociation from IKK2

Parameter('a_2f', 1.35)
Parameter('a_2r', 0.075)
Parameter('b_kf', 0.36)
Parameter('b_2r', 0.105)
Parameter('e_2r', 0.105)
Parameter('d_1r', 0.105)
Parameter('e_2f', 0.54)
Parameter('d_1f', 0.54)
Rule('a2_adc', IkBa(ikk=None, nfkb=None, S='C') + IKK2(ikb=None, S='C') <> IkBa(ikk=1, nfkb=None, S='C')%IKK2(ikb=1, S='C'), a_2f, a_2r)
Rule('b2_adc', IkBb(ikk=None, nfkb=None, S='C') + IKK2(ikb=None, S='C') <> IkBb(ikk=1, nfkb=None, S='C')%IKK2(ikb=1, S='C'), b_kf, b_2r)
Rule('e2_adc', IkBe(ikk=None, nfkb=None, S='C') + IKK2(ikb=None, S='C') <> IkBe(ikk=1, nfkb=None, S='C')%IKK2(ikb=1, S='C'), e_2f, e_2r)
Rule('d1_adc', IkBd(ikk=None, nfkb=None, S='C') + IKK1(ikb=None, S='C') <> IkBd(ikk=1, nfkb=None, S='C')%IKK1(ikb=1, S='C'), d_1f, d_1r)

Rule('d1_adc', IkBd(ikk=None, nfkb=None, S='C') + IKK1(ikb=None, S='C') <> IkBd(ikk=1, nfkb=None, S='C') % IKK1(ikb=1, S='C'), d_1f, d_1r)

In [11]:
Parameter('IkB_IKKf', 30)
Parameter('IkB_IKKr', 6e-5)
Rule('a2n_c1', IkBa(ikk=1, nfkb=None, S='C')%IKK2(ikb=1, S='C') + NFkB(ikb=None, S='C') <> IkBa(ikk=1, nfkb=2, S='C')%IKK2(ikb=1, S='C')%NFkB(ikb=2, S='C'), IkB_IKKf, IkB_IKKr)
Rule('b2n_c1', IkBb(ikk=1, nfkb=None, S='C')%IKK2(ikb=1, S='C') + NFkB(ikb=None, S='C') <> IkBb(ikk=1, nfkb=2, S='C')%IKK2(ikb=1, S='C')%NFkB(ikb=2, S='C'), IkB_IKKf, IkB_IKKr)
Rule('e2n_c1', IkBe(ikk=1, nfkb=None, S='C')%IKK2(ikb=1, S='C') + NFkB(ikb=None, S='C') <> IkBe(ikk=1, nfkb=2, S='C')%IKK2(ikb=1, S='C')%NFkB(ikb=2, S='C'), IkB_IKKf, IkB_IKKr)
Rule('d1n_c1', IkBd(ikk=1, nfkb=None, S='C')%IKK1(ikb=1, S='C') + NFkB(ikb=None, S='C') <> IkBd(ikk=1, nfkb=2, S='C')%IKK1(ikb=1, S='C')%NFkB(ikb=2, S='C'), IkB_IKKf, IkB_IKKr)


Parameter('an_2f', 11.1)
Parameter('bn_2f', 2.88)
Parameter('en_2f', 4.2)
Parameter('dn_1f', 4.2)
Parameter('an_2r', 0.075)
Parameter('bn_2r', 0.105)
Parameter('en_2r', 0.105)
Parameter('dn_1r', 0.105)
Rule('a2n_c', IkBa(ikk=None, nfkb=1, S='C')%NFkB(ikb=1, S='C') + IKK2(ikb=None, S='C') <> IkBa(ikk=2, nfkb=1, S='C')%NFkB(ikb=1, S='C')%IKK2(ikb=2, S='C'), an_2f, an_2r)
Rule('b2n_c', IkBb(ikk=None, nfkb=1, S='C')%NFkB(ikb=1, S='C') + IKK2(ikb=None, S='C') <> IkBb(ikk=2, nfkb=1, S='C')%NFkB(ikb=1, S='C')%IKK2(ikb=2, S='C'), bn_2f, bn_2r)
Rule('e2n_c', IkBe(ikk=None, nfkb=1, S='C')%NFkB(ikb=1, S='C') + IKK2(ikb=None, S='C') <> IkBe(ikk=2, nfkb=1, S='C')%NFkB(ikb=1, S='C')%IKK2(ikb=2, S='C'), en_2f, en_2r)
Rule('d1n_c', IkBd(ikk=None, nfkb=1, S='C')%NFkB(ikb=1, S='C') + IKK1(ikb=None, S='C') <> IkBd(ikk=2, nfkb=1, S='C')%NFkB(ikb=1, S='C')%IKK1(ikb=2, S='C'), dn_1f, dn_1r)


Rule('d1n_c', IkBd(ikk=None, nfkb=1, S='C') % NFkB(ikb=1, S='C') + IKK1(ikb=None, S='C') <> IkBd(ikk=2, nfkb=1, S='C') % NFkB(ikb=1, S='C') % IKK1(ikb=2, S='C'), dn_1f, dn_1r)

In [12]:
# Parameter('IkB_IKKf', 30)
# Parameter('IkB_IKKr', 6e-5)
Rule('an_adc', IkBa(ikk=None, nfkb=None, S='C') + NFkB(ikb=None, S='C') <> IkBa(ikk=None, nfkb=1, S='C')%NFkB(ikb=1, S='C'), IkB_IKKf, IkB_IKKr)
Rule('bn_adc', IkBb(ikk=None, nfkb=None, S='C') + NFkB(ikb=None, S='C') <> IkBb(ikk=None, nfkb=1, S='C')%NFkB(ikb=1, S='C'), IkB_IKKf, IkB_IKKr)
Rule('en_adc', IkBe(ikk=None, nfkb=None, S='C') + NFkB(ikb=None, S='C') <> IkBe(ikk=None, nfkb=1, S='C')%NFkB(ikb=1, S='C'), IkB_IKKf, IkB_IKKr)
Rule('dn_adc', IkBd(ikk=None, nfkb=None, S='C') + NFkB(ikb=None, S='C') <> IkBd(ikk=None, nfkb=1, S='C')%NFkB(ikb=1, S='C'), IkB_IKKf, IkB_IKKr)

Rule('an_adn', IkBa(ikk=None, nfkb=None, S='N') + NFkB(ikb=None, S='N') <> IkBa(ikk=None, nfkb=1, S='N')%NFkB(ikb=1, S='N'), IkB_IKKf, IkB_IKKr)
Rule('bn_adn', IkBb(ikk=None, nfkb=None, S='N') + NFkB(ikb=None, S='N') <> IkBb(ikk=None, nfkb=1, S='N')%NFkB(ikb=1, S='N'), IkB_IKKf, IkB_IKKr)
Rule('en_adn', IkBe(ikk=None, nfkb=None, S='N') + NFkB(ikb=None, S='N') <> IkBe(ikk=None, nfkb=1, S='N')%NFkB(ikb=1, S='N'), IkB_IKKf, IkB_IKKr)
Rule('dn_adn', IkBd(ikk=None, nfkb=None, S='N') + NFkB(ikb=None, S='N') <> IkBd(ikk=None, nfkb=1, S='N')%NFkB(ikb=1, S='N'), IkB_IKKf, IkB_IKKr)

Rule('dn_adn', IkBd(ikk=None, nfkb=None, S='N') + NFkB(ikb=None, S='N') <> IkBd(ikk=None, nfkb=1, S='N') % NFkB(ikb=1, S='N'), IkB_IKKf, IkB_IKKr)

In [13]:
#IkB and NFkB cellular localization reactions

Parameter('af', 0.09)
Parameter('bf', 0.009)
Parameter('ef', 0.045)
Parameter('df', 0.045)
Parameter('ancf', 0.012)
Parameter('bncf', 0.012)
Parameter('encf', 0.012)
Parameter('dncf', 0.012)
Rule('a_nc', IkBa(ikk=None, nfkb=None, S='C') <> IkBa(ikk=None, nfkb=None, S='N'), af, ancf)
Rule('b_nc', IkBb(ikk=None, nfkb=None, S='C') <> IkBb(ikk=None, nfkb=None, S='N'), bf, bncf)
Rule('e_nc', IkBe(ikk=None, nfkb=None, S='C') <> IkBe(ikk=None, nfkb=None, S='N'), ef, encf)
Rule('d_nc', IkBd(ikk=None, nfkb=None, S='C') <> IkBd(ikk=None, nfkb=None, S='N'), ef, dncf)

Parameter('anf', 0.276)
Parameter('bnf', 0.0276)
Parameter('enf', 0.138)
Parameter('dnf', 0.276)
Parameter('anr', 0.828)
Parameter('bnr', 0.414)
Parameter('enr', 0.414)
Parameter('dnr', 0.414)
Rule('an_nc', IkBa(ikk=None, nfkb=1, S='C')%NFkB(ikb=1, S='C') <> IkBa(ikk=None, nfkb=1, S='N')%NFkB(ikb=1, S='N'), anf, anr)
Rule('bn_nc', IkBb(ikk=None, nfkb=1, S='C')%NFkB(ikb=1, S='C') <> IkBb(ikk=None, nfkb=1, S='N')%NFkB(ikb=1, S='N'), bnf, bnr)
Rule('en_nc', IkBe(ikk=None, nfkb=1, S='C')%NFkB(ikb=1, S='C') <> IkBe(ikk=None, nfkb=1, S='N')%NFkB(ikb=1, S='N'), enf, enr)
Rule('dn_nc', IkBd(ikk=None, nfkb=1, S='C')%NFkB(ikb=1, S='C') <> IkBd(ikk=None, nfkb=1, S='N')%NFkB(ikb=1, S='N'), dnf, dnr)

Parameter('nf', 5.4)
Parameter('nr', 0.0048)
Rule('n_nc', NFkB(ikb=None, S='C') <> NFkB(ikb=None, S='N'), nf, nr)


Rule('n_nc', NFkB(ikb=None, S='C') <> NFkB(ikb=None, S='N'), nf, nr)

In [14]:
# IkB Protein Degradation Reactions

Parameter('a_dc', 0.12)
Parameter('b_dc', 0.18)
Parameter('e_dc', 0.18)
Parameter('d_dc', 0.0014)
Parameter('a_dn', 0.12)
Parameter('b_dn', 0.18)
Parameter('e_dn', 0.18)
Parameter('d_dn', 0.0014)

Rule('ad_c', IkBa(ikk=None, nfkb=None, S='C') >> None, a_dc)
Rule('bd_c', IkBb(ikk=None, nfkb=None, S='C') >> None, b_dc)
Rule('ed_c', IkBe(ikk=None, nfkb=None, S='C') >> None, e_dc)
Rule('dd_c', IkBd(ikk=None, nfkb=None, S='C') >> None, d_dc)

Rule('ad_n', IkBa(ikk=None, nfkb=None, S='N') >> None, a_dn)
Rule('bd_n', IkBb(ikk=None, nfkb=None, S='N') >> None, b_dn)
Rule('ed_n', IkBe(ikk=None, nfkb=None, S='N') >> None, e_dn)
Rule('dd_n', IkBd(ikk=None, nfkb=None, S='N') >> None, d_dn)

Parameter('c_bn', 0.00006)
Parameter('n_bn', 0.00006)

Rule('an_c', IkBa(ikk=None, nfkb=1, S='C')%NFkB(ikb=1, S='C') >> NFkB(ikb=None, S='C'), c_bn)
Rule('bn_c', IkBb(ikk=None, nfkb=1, S='C')%NFkB(ikb=1, S='C') >> NFkB(ikb=None, S='C'), c_bn)
Rule('en_c', IkBe(ikk=None, nfkb=1, S='C')%NFkB(ikb=1, S='C') >> NFkB(ikb=None, S='C'), c_bn)
Rule('dn_c', IkBd(ikk=None, nfkb=1, S='C')%NFkB(ikb=1, S='C') >> NFkB(ikb=None, S='C'), c_bn)

Rule('an_n', IkBa(ikk=None, nfkb=1, S='N')%NFkB(ikb=1, S='N') >> NFkB(ikb=None, S='N'), n_bn)
Rule('bn_n', IkBb(ikk=None, nfkb=1, S='N')%NFkB(ikb=1, S='N') >> NFkB(ikb=None, S='N'), n_bn)
Rule('en_n', IkBe(ikk=None, nfkb=1, S='N')%NFkB(ikb=1, S='N') >> NFkB(ikb=None, S='N'), n_bn)
Rule('dn_n', IkBd(ikk=None, nfkb=1, S='N')%NFkB(ikb=1, S='N') >> NFkB(ikb=None, S='N'), n_bn)

Parameter('a2_2', 0.0018)
Parameter('b2_2', 0.0006)
Parameter('ed21_21', 0.0012)

Rule('a2_c', IkBa(ikk=1, nfkb=None, S='C')%IKK2(ikb=1, S='C') >> IKK2(ikb=None, S='C'), a2_2)
Rule('b2_c', IkBb(ikk=1, nfkb=None, S='C')%IKK2(ikb=1, S='C') >> IKK2(ikb=None, S='C'), b2_2)
Rule('e2_c', IkBe(ikk=1, nfkb=None, S='C')%IKK2(ikb=1, S='C') >> IKK2(ikb=None, S='C'), ed21_21)
Rule('d1_c', IkBd(ikk=1, nfkb=None, S='C')%IKK1(ikb=1, S='C') >> IKK1(ikb=None, S='C'), ed21_21)

Parameter('an2_2n', 0.36)
Parameter('bn2_2n', 0.12)
Parameter('edn21_21n', 0.18)

Rule('a2n_2nc', IkBa(ikk=2, nfkb=1, S='C')%NFkB(ikb=1, S='C')%IKK2(ikb=2, S='C') >> IKK2(ikb=None, S='C') + NFkB(ikb=None, S='C'), an2_2n)
Rule('b2n_2nc', IkBb(ikk=2, nfkb=1, S='C')%NFkB(ikb=1, S='C')%IKK2(ikb=2, S='C') >> IKK2(ikb=None, S='C') + NFkB(ikb=None, S='C'), bn2_2n)
Rule('e2n_2nc', IkBe(ikk=2, nfkb=1, S='C')%NFkB(ikb=1, S='C')%IKK2(ikb=2, S='C') >> IKK2(ikb=None, S='C') + NFkB(ikb=None, S='C'), edn21_21n)
Rule('d1n_1nc', IkBd(ikk=2, nfkb=1, S='C')%NFkB(ikb=1, S='C')%IKK1(ikb=2, S='C') >> IKK1(ikb=None, S='C') + NFkB(ikb=None, S='C'), edn21_21n)


Rule('d1n_1nc', IkBd(ikk=2, nfkb=1, S='C') % NFkB(ikb=1, S='C') % IKK1(ikb=2, S='C') >> IKK1(ikb=None, S='C') + NFkB(ikb=None, S='C'), edn21_21n)

In [15]:
generate_network(model)
generate_equations(model)

# for i,ode in enumerate(model.odes):
#      print i,":",ode

# for i,obs in enumerate(model.observables):
#      print i,":",obs

for i,sp in enumerate(model.species):
     print i,":",sp
#
# # for i,reactions in enumerate(model.reactions):
# #      print i,":",reactions
# #
# for i,parameters in enumerate(model.parameters):
#      print i,":",parameters
if __name__ == "__main__":
    tspan = np.linspace(0, 720, 721)
    print len(tspan)
    solver = Solver(model,tspan,verbose=True)
    generate_equations(model, verbose = True)


equil = odesolve(model, np.linspace(-4000, 1, 4001), verbose=True)
plt.figure(1)
plt.plot( np.linspace(-4000, 1, 4001), equil["NFkBn_free"], label = NFkBn_free.name)
plt.legend(loc = 0)

# plt.figure(2)
# plt.plot( np.linspace(-4000, 1, 4001), equil["IKK2_obs"], label = IKK2_obs.name)
# plt.legend(loc = 0)

x = np.array([0, 5, 10, 15, 20, 25, 30, 260, 480, 2880], dtype = float)
y1 = np.array([0.01, 0.6, 1, 0.65, 0.5, 0.36, 0.2, 0.1, 0.03, 0.01], dtype = float)
pchip_obj1 = scipy.interpolate.PchipInterpolator(x, y1)(tspan)
# print(pchip_obj1)
# exit()
# xi = np.arange(x[0],x[-1],0.01)
# yi1 = pchip_obj1(xi)


last_conc = [equil[x][-1] for x in ['__s%d' %i for i in np.arange(len(model.species))]] #setting previous conc species equal to none
# yobs = []
nsims = len(tspan) - 1 #simulation time is 720
# yobs = np.empty((nsims, len(model.observables)))
rows = nsims + 1 # nSims
cols = len(model.observables) #length of observables
array = np.empty((rows,cols)) #creating empty array of R and C
yobs = pd.DataFrame(array,columns=[str(i) for i in model.observables])
# yobs = pd.DataFrame(array,columns=[str(i) for i in model.observables])
# print yobs
# yobs = pd.DataFrame(array,columns=['observable'+str(a) for a in range(cols)]) #creating panda data frame for labeling of array

# print (yobs.shape)
# yobs = pd.DataFrame(index = nsims, columns = len(model.observables))
model.parameters['IKK2_0'].value = pchip_obj1[0]
assert model.observables[1].name == 'IKK2_obs'
assert str(model.species[1]) == "IKK2(ikb=None, S='C')"
for i in range(nsims):
    print(i) #printing each simulation iteration
    solver.tspan = [tspan[i], tspan[i+1]] #simulating in 2 step intervals
    solver.run(y0 = last_conc) #setting initial concentrations to previous simulation value
    last_conc = solver.y[1,:] # running solver taking first row and all simulation observables
    last_conc[1] = pchip_obj1[i] #updating IKK2 interpolated values over each simulation
    if i == 0:
        array[0,:] = solver.yobs_view[0]
    array[i+1 ,:] = solver.yobs_view[1] #taking each simulation index and all obs and keeping the last obs in last time point of simulation
    # print(solver.yobs_view[1])
    # yobs.append(solver.yobs_view[1][1])
# print yobs

plt.figure(1)
plt.plot(tspan, array[:,0], label= NFkBn_free.name)
plt.xlabel("Time (in minutes)", fontsize=16)
plt.ylabel("Concentration", fontsize=16)
plt.legend(loc=0)
plt.show()

0 : IKK1(ikb=None, S='C')
1 : IKK2(ikb=None, S='C')
2 : NFkB(ikb=None, S='N')
3 : __source()
4 : IkBa_mRNA()
5 : IkBb_mRNA()
6 : IkBe_mRNA()
7 : IkBd_mRNA()
8 : NFkB(ikb=None, S='C')
9 : __sink()
10 : IkBa(ikk=None, nfkb=None, S='C')
11 : IkBb(ikk=None, nfkb=None, S='C')
12 : IkBe(ikk=None, nfkb=None, S='C')
13 : IkBd(ikk=None, nfkb=None, S='C')
14 : IKK2(ikb=1, S='C') % IkBa(ikk=1, nfkb=None, S='C')
15 : IKK2(ikb=1, S='C') % IkBb(ikk=1, nfkb=None, S='C')
16 : IKK2(ikb=1, S='C') % IkBe(ikk=1, nfkb=None, S='C')
17 : IKK1(ikb=1, S='C') % IkBd(ikk=1, nfkb=None, S='C')
18 : IkBa(ikk=None, nfkb=1, S='C') % NFkB(ikb=1, S='C')
19 : IkBb(ikk=None, nfkb=1, S='C') % NFkB(ikb=1, S='C')
20 : IkBe(ikk=None, nfkb=1, S='C') % NFkB(ikb=1, S='C')
21 : IkBd(ikk=None, nfkb=1, S='C') % NFkB(ikb=1, S='C')
22 : IkBa(ikk=None, nfkb=None, S='N')
23 : IkBb(ikk=None, nfkb=None, S='N')
24 : IkBe(ikk=None, nfkb=None, S='N')
25 : IkBd(ikk=None, nfkb=None, S='N')
26 : IKK2(ikb=1, S='C') % IkBa(ikk=1, nfkb=2, S='C')

AttributeError: 'Solver' object has no attribute 'yobs_view'